# Nestlé HR Assistant Setup

1. Load Nestlé HR PDF  
2. Split into chunks with PyPDFLoader  
3. Create embeddings & vectorstore  
4. Build QA retrieval chain  
5. Launch Gradio interface


# Situation

Nestlé, as a leading multinational corporation, manages thousands of employees globally and relies on clear HR policies to ensure consistency and efficiency. However, HR staff often spend valuable time searching through long policy documents to answer employee questions.  
To address this, I have been tasked with creating an AI-powered HR assistant that can quickly retrieve accurate information from Nestlé’s HR Policy (2012 edition). This assistant should streamline HR workflows and provide employees with fast, reliable answers.


In [1]:
import os

# Ensure we’re at the project root
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print("Working directory:", os.getcwd())

# Show what’s in data/raw
raw_dir = os.path.join(os.getcwd(), "data", "raw")
print("data/raw contains:", os.listdir(raw_dir))

from langchain.document_loaders import PyPDFLoader

# Adjust this to match the exact filename you see above
pdf_filename = "the_nestle_hr_policy_pdf_2012.pdf"
pdf_path = os.path.join(raw_dir, pdf_filename)
print("Loading:", pdf_path)

loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages")



Working directory: /Users/sheilamcgovern/Desktop/Projects2025/nestle_hr_assistant
data/raw contains: ['the_nestle_hr_policy_pdf_2012.pdf', '.ipynb_checkpoints']
Loading: /Users/sheilamcgovern/Desktop/Projects2025/nestle_hr_assistant/data/raw/the_nestle_hr_policy_pdf_2012.pdf
Loaded 8 pages


# Task

My task is to develop a conversational chatbot that can answer employee questions about Nestlé’s HR Policy (2012).  
The chatbot should:
- Extract and process the PDF policy document.
- Represent the content in a format that supports fast, accurate retrieval.
- Use OpenAI’s GPT model to provide well-structured answers.
- Provide a user-friendly interface through Gradio.
- Clearly cite sources (page numbers) when giving responses.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import os

# Assume `docs` is loaded from PDF (execution_count 1)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunks = splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

# Deduplicate and fix page numbers
chunk_data = []
seen_content = set()
for i, c in enumerate(chunks):
    content = c.page_content.replace("\n", " ").strip()
    if content not in seen_content:
        seen_content.add(content)
        chunk_data.append({
            "chunk_id": i,
            "page": c.metadata.get("page") + 1,
            "text": content
        })
df = pd.DataFrame(chunk_data)
print(f"Unique chunks: {len(df)}")

# Save to CSV
os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/hr_policy_chunks.csv", index=False)
print("CSV written.")

Split into 60 chunks
Unique chunks: 60
CSV written.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print("API Key loaded:", "Yes" if api_key else "No")
if api_key:
    print("API Key preview:", api_key[:5] + "...")

API Key loaded: Yes
API Key preview: sk-pr...


In [3]:
import os
os.environ["CHROMA_DISABLE_TELEMETRY"] = "1"
from dotenv import load_dotenv
load_dotenv()

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import pandas as pd
from langchain.schema import Document

# Load chunks
df = pd.read_csv("data/processed/hr_policy_chunks.csv")
docs = [Document(page_content=row["text"], metadata={"page": int(row["page"]), "chunk_id": int(row["chunk_id"])}) for _, row in df.iterrows()]
print(f"Loaded {len(docs)} docs")

# Create vector store
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectordb = Chroma(collection_name="hr_policy_2012", embedding_function=embeddings, persist_directory="db/chroma")
# Clear any existing data
if vectordb.get()['ids']:
    vectordb.delete_collection()
    vectordb = Chroma(collection_name="hr_policy_2012", embedding_function=embeddings, persist_directory="db/chroma")
# Add unique documents with unique IDs
unique_docs = list({doc.page_content: doc for doc in docs}.values())
ids = [f"doc_{i}" for i in range(len(unique_docs))]  # Unique IDs
vectordb.add_documents(documents=unique_docs, ids=ids)
print(f"Stored {len(unique_docs)} unique docs")

# Verify content
docs = vectordb.get()
print(f"Docs in vector store: {len(docs['ids'])}")
if docs['ids']:
    print("Sample metadata:", docs["metadatas"][0])
    print("Sample content:", docs["documents"][0][:100], "...")

Loaded 60 docs


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


Stored 60 unique docs
Docs in vector store: 60
Sample metadata: {'chunk_id': 0, 'page': 1}
Sample content: Policy Mandatory September  2012 The Nestlé   Human Resources Policy ...


# Action



1. **Imported Tools and Set Up Environment**  
   - Configured Python, installed dependencies, and set up OpenAI API keys.

2. **Loaded and Split Nestlé HR Policy PDF**  
   - Used `PyPDFLoader` to load the 2012 HR policy document.
   - Split the document into overlapping text chunks using `RecursiveCharacterTextSplitter` for better retrieval accuracy.

3. **Created Vector Embeddings**  
   - Converted chunks into numerical embeddings with OpenAI’s `text-embedding-ada-002`.
   - Stored these in a ChromaDB vector store for semantic search.

4. **Built a Question-Answering Chain**  
   - Used `RetrievalQA` with `gpt-3.5-turbo` to generate context-aware answers.
   - Designed a `PromptTemplate` requiring direct quotes and fallback responses when the document lacks relevant info.

5. **Designed User Interface with Gradio**  
   - Implemented a chatbot where users can type queries and receive answers with cited page references.


In [24]:
import os
os.environ["CHROMA_DISABLE_TELEMETRY"] = "1"
from dotenv import load_dotenv
load_dotenv()

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import gradio as gr
import pandas as pd
import traceback

# Load chunks
df = pd.read_csv("data/processed/hr_policy_chunks.csv")
docs = [
    Document(
        page_content=row["text"], 
        metadata={"page": int(row["page"]), "chunk_id": int(row["chunk_id"])}
    )
    for _, row in df.iterrows()
]
print(f"Loaded {len(docs)} docs")

# Create vector store
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectordb = Chroma(
    collection_name="hr_policy_2012",
    embedding_function=embeddings,
    persist_directory="db/chroma"
)
unique_docs = list({doc.page_content: doc for doc in docs}.values())
ids = [f"doc_{i}" for i in range(len(unique_docs))]
vectordb.add_documents(documents=unique_docs, ids=ids)
print(f"Stored {len(unique_docs)} docs")

# Prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Answer using only the provided snippets. Quote exact text relevant to the question. "
        "If the snippets do not directly address the question, say: "
        "'I don’t know based on the 2012 HR Policy document.'\n\n"
        "Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    )
)

# QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# Respond function for Gradio
def respond(message, history=[]):
    try:
        print(f"Processing query: {message}")
        if not message:
            return history, "Please enter a question."

        result = qa_chain.invoke({"query": message})
        answer = str(result["result"])

        if "I don’t know" not in answer.lower():
            source_docs = result["source_documents"]
            seen = set()
            unique_sources = [
                doc for doc in source_docs
                if not (doc.metadata["chunk_id"] in seen or seen.add(doc.metadata["chunk_id"]))
            ]
            if unique_sources:
                pages = sorted({doc.metadata["page"] for doc in unique_sources})
                answer += f"\n\n*Source: Page{'s' if len(pages) > 1 else ''} {', '.join(map(str, pages))}*"
        else:
            answer = (
                "I don’t know based on the 2012 HR Policy document.\n\n"
                "*Note: The 2012 policy lacks specific details on this topic. "
                "Contact Nestlé HR for current policies.*"
            )

        # Format for Gradio Chatbot with type="messages"
        history = history + [
            {"role": "user", "content": str(message)},
            {"role": "assistant", "content": answer}
        ]

        print(f"Returning history: {len(history)} messages")
        return history, ""

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        traceback.print_exc()
        history = history + [
            {"role": "user", "content": str(message)},
            {"role": "assistant", "content": error_msg}
        ]
        return history, ""

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Nestlé HR Assistant\nAsk about the 2012 HR Policy. Example: 'What are the total rewards?'")
    chatbot = gr.Chatbot(type="messages")
    txt = gr.Textbox(show_label=False, placeholder="Type question and hit enter")
    txt.submit(respond, [txt, chatbot], [chatbot, txt])

demo.launch()


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Loaded 60 docs
Stored 60 docs
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Processing query: what are total rewards?
Returning history: 2 messages
Processing query: what are working hours
Returning history: 4 messages
Processing query: how do I handle paid time off?
Returning history: 6 messages


In [5]:
import gradio
print("Gradio version:", gradio.__version__)

Gradio version: 5.36.2


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectordb = Chroma(collection_name="hr_policy_2012", embedding_function=embeddings, persist_directory="db/chroma")
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

for query in ["total rewards", "working hours"]:
    print(f"\nQuery: {query}")
    docs = retriever.invoke(query)
    for d in docs:
        print(f"Page {d.metadata['page']}, Chunk {d.metadata['chunk_id']}: {d.page_content[:100]}...")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Query: total rewards


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Page 5, Chunk 19: value and trust that our name brings to those  who work with us; the relationships with our line  ma...
Page 5, Chunk 20: receive. Nestlé, therefore, focuses on Fixed Pay,  Variable Pay, Benefits, Personal Growth and  Deve...
Page 5, Chunk 22: Nestlé Total Rewards programmes must be  established within the social and legal framework  of each ...
Page 5, Chunk 24: transparency. Corporate policy:  Nestlé Total Rewards Policy We are committed to providing our emplo...
Page 5, Chunk 18: The Nestlé Human Resources Policy 3  Total rewards Attracting new hires and keeping current  employe...

Query: working hours
Page 5, Chunk 30: employees is heard. Corporate policy:  Policy on Conditions of Work and Employment  Employment and w...
Page 5, Chunk 28: and we insist that they also take steps so that  adequate working conditions are made available  to ...
Page 7, Chunk 50: communication is established in the workplace.  While dialogue with trade unions is essential, it  d...
Pa

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectordb = Chroma(collection_name="hr_policy_2012", embedding_function=embeddings, persist_directory="db/chroma")
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

for query in ["total rewards", "working hours"]:
    print(f"\nQuery: {query}")
    docs = retriever.invoke(query)
    for d in docs:
        print(f"Page {d.metadata['page']}, Chunk {d.metadata['chunk_id']}: {d.page_content[:100]}...")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Query: total rewards
Page 5, Chunk 19: value and trust that our name brings to those  who work with us; the relationships with our line  ma...
Page 5, Chunk 20: receive. Nestlé, therefore, focuses on Fixed Pay,  Variable Pay, Benefits, Personal Growth and  Deve...
Page 5, Chunk 22: Nestlé Total Rewards programmes must be  established within the social and legal framework  of each ...
Page 5, Chunk 24: transparency. Corporate policy:  Nestlé Total Rewards Policy We are committed to providing our emplo...
Page 5, Chunk 18: The Nestlé Human Resources Policy 3  Total rewards Attracting new hires and keeping current  employe...

Query: working hours
Page 5, Chunk 30: employees is heard. Corporate policy:  Policy on Conditions of Work and Employment  Employment and w...
Page 5, Chunk 28: and we insist that they also take steps so that  adequate working conditions are made available  to ...
Page 7, Chunk 50: communication is established in the workplace.  While dialogue with trade unions is 

In [7]:
from langchain_openai import ChatOpenAI
import traceback

try:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = llm.invoke("Test query: Say 'Hello'")
    print("API Response:", response.content)
except Exception as e:
    print(f"OpenAI API error: {str(e)}")
    traceback.print_exc()

API Response: Hello! How can I assist you today?


# Demonstration: Sample Query

Below is an example query run through the question-answering chain, showing how the chatbot retrieves an answer and cites the HR policy document. This demonstrates the system’s functionality even outside the Gradio interface.


In [8]:
# Example query for demonstration
try:
    sample_question = "What are the total rewards mentioned in the HR policy?"
    print(f"User Question: {sample_question}\n")

    result = qa_chain.invoke({"query": sample_question})
    print("Chatbot Answer:")
    print(result["result"])

    # Show source documents/pages
    source_docs = result.get("source_documents", [])
    if source_docs:
        pages = sorted({doc.metadata["page"] for doc in source_docs})
        print(f"\nSource Pages: {', '.join(map(str, pages))}")
    else:
        print("\nNo source documents returned.")
except Exception as e:
    print("Error during sample query:", str(e))


User Question: What are the total rewards mentioned in the HR policy?

Chatbot Answer:
Nestlé focuses on Fixed Pay, Variable Pay, Benefits, Personal Growth and Development and Work Life Environment as the key elements that define Total Rewards.

Source Pages: 5


# Demonstration: Fallback Query

 - The chatbot is designed to admit when the HR policy does not contain the requested information.  
 - This prevents misinformation and directs users to contact HR for topics not covered in the 2012 policy.  
 - Below is an example of such a query.


In [10]:
# Example query that likely won't be in the 2012 HR Policy
try:
    fallback_question = "Does the HR policy mention remote work guidelines?"
    print(f"User Question: {fallback_question}\n")

    result = qa_chain.invoke({"query": fallback_question})
    print("Chatbot Answer:")
    print(result["result"])

    # Show source docs if any (there might be none)
    source_docs = result.get("source_documents", [])
    if source_docs:
        pages = sorted({doc.metadata["page"] for doc in source_docs})
        print(f"\nSource Pages: {', '.join(map(str, pages))}")
    else:
        print("\nNo relevant source documents returned (as expected).")
except Exception as e:
    print("Error during fallback query:", str(e))


User Question: Does the HR policy mention remote work guidelines?

Chatbot Answer:
I don’t know based on the 2012 HR Policy document.

Source Pages: 1, 5


# Result

The completed project delivers a functional **AI-powered HR Assistant** that answers questions from Nestlé’s 2012 HR Policy.  
- **User Experience:** Employees can ask free-form questions in a Gradio chatbot and receive clear, concise answers.  
- **Accuracy:** Responses are generated using semantic retrieval from the policy document, ensuring relevant and sourced information.  
- **Transparency:** The chatbot cites the exact policy page(s) for each answer, helping employees trust the results.  
- **Fallback Handling:** If the document does not contain the answer, the chatbot informs the user and suggests contacting HR for current policies.  

This assistant demonstrates how conversational AI can improve HR efficiency by reducing manual search time and providing a reliable first point of contact for policy-related queries.


# Final Demonstration Summary

## **Nestlé HR Assistant** according to the project requirements.

### Demonstration Outcomes
- **Sample Query (Success)**  
  - Question: *What are the total rewards mentioned in the HR policy?*  
  - The assistant retrieved the correct answer from the 2012 policy and cited the relevant page(s).  
- **Fallback Query (Graceful Handling)**  
  - Question: *Does the HR policy mention remote work guidelines?*  
  - The assistant responded responsibly with *I don’t know based on the 2012 HR Policy document*, reminding users to contact HR for more current information.

### Key Features Implemented
- Loaded and processed Nestlé’s 2012 HR Policy PDF.  
- Split the text into manageable chunks for semantic search.  
- Generated embeddings with `text-embedding-ada-002` and stored them in **ChromaDB**.  
- Built a **RetrievalQA** chain using `gpt-3.5-turbo`.  
- Created a **Gradio chatbot interface** for interactive use.  
- Added logic for **accurate citations** and **fallback responses**.

### Conclusion
The HR Assistant demonstrates how conversational AI can enhance HR operations by:
- Reducing time spent searching policy documents.  
- Providing reliable, cited answers.  
- Handling out-of-scope questions responsibly.  

This fulfills the **Situation, Task, Action, and Result** framework and meets the project’s goals.

